In [1]:
import random
import torch
import math


class DynamicNet(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate five parameters and assign them as members.
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 4, 5
        and reuse the e parameter to compute the contribution of these orders.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same parameter many
        times when defining a computational graph.
        """
        y = self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
        for exp in range(4, random.randint(4, 6)):
            y = y + self.e * x ** exp
        return y

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
model = DynamicNet()

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
for t in range(30000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 2000 == 1999:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

1999 3507.241455078125
3999 1627.653564453125
5999 794.85791015625
7999 403.08258056640625
9999 198.22640991210938
11999 104.01927185058594
13999 55.05368423461914
15999 31.630266189575195
17999 20.212478637695312
19999 14.45259952545166
21999 11.642106056213379
23999 10.217216491699219
25999 9.507014274597168
27999 9.179034233093262
29999 8.996671676635742
Result: y = -0.01326314453035593 + 0.8596036434173584 x + 0.001880655880086124 x^2 + -0.09392840415239334 x^3 + 7.896095485193655e-05 x^4 ? + 7.896095485193655e-05 x^5 ?


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

class testModule(nn.Module):

    def __init__(self):
        super(testModule, self).__init__()
        self.fc1 = nn.Linear(5, 10, bias=True)
        self.fc2 = nn.Linear(10, 10, bias=False)

        # Remove the weights as we override them in the forward
        # so that they don't show up when calling .parameters()
        del self.fc1.weight
        del self.fc2.weight

        self.fc2_base_weights = nn.Parameter(torch.randn(10, 10))
        self.shared_weights = nn.Parameter(torch.randn(10, 5))

    def forward(self, x):
        # Update the weights
        index = [1, 3, 5, 7, 9]
        self.fc1.weight = self.shared_weights
        self.fc2.weight = self.fc2_base_weights.clone()
        self.fc2.weight[:, index] = self.shared_weights

        x = self.fc1(x)
        x = self.fc2(x)
        return x

    # def _weight(self):

In [5]:
# sharing weights between the units of the same layer and between layers 
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.w = nn.Parameter(torch.rand(3))  # w1, w2, w3
    def forward(self, x):
        x = torch.sum(x * self.w)
        x = torch.sum(x * self.w)
        return x
# Net()

Net()

In [10]:
import torch.nn as nn
b= 3
n=1000
input=torch.rand(b,n,n)

first_layer_node_1=nn.ReLU(w1*w2*w3*input[0])# n by n
first_layer_node_2=nn.ReLU(w1*w2*w3*input[1])# n by n
first_layer_node_3=nn.ReLU(w1*w2*w3*input[2])# n by n


output_layer=nn.ReLU(w1*first_layer_node_1+w2*first_layer_node_2+w3*first_layer_node_3)# n by n

NameError: name 'w1' is not defined

In [11]:
import torch
import torch.autograd as autograd
import torch.nn as nn

BATCH_SIZE = 64
DIM = 128
SEQ_LEN = 12
torch.set_default_tensor_type('torch.cuda.FloatTensor')

class ResBlock(nn.Module):

    def __init__(self):
        super(ResBlock, self).__init__()
        self.res_block = nn.Sequential(
            nn.ReLU(True),
            nn.Conv1d(DIM, DIM, 5, padding=2),#nn.Linear(DIM, DIM),
            nn.ReLU(True),
            nn.Conv1d(DIM, DIM, 5, padding=2),#nn.Linear(DIM, DIM),
        )

    def forward(self, input):
        output = self.res_block(input)
        return input + (0.3*output)

class Discriminator(nn.Module):

    def __init__(self, charmap1, charmap2):
        super(Discriminator, self).__init__()
        self.conv0_a = nn.Conv1d(len(charmap1), DIM, 1)
        self.conv0_b = nn.Conv1d(len(charmap2), DIM, 1)
        self.block1_a = ResBlock()
        self.block1_b = ResBlock()
        self.base2 = ResBlock()
        self.base3 = ResBlock()
        self.base4 = ResBlock()
        self.base5 = ResBlock()
        self.linear = nn.Linear(SEQ_LEN*DIM, 1)

    def forward(self, x_a, x_b):
        output_a = self.block1_a(self.conv0_a(x_a.transpose(1, 2))) 
        output_b = self.block1_b(self.conv0_b(x_b.transpose(1, 2)))
        #output = torch.cat((output_a, output_b), 0)
        output_a = self.base2(output_a)
        output_b = self.base2(output_b)
        output_a = self.base3(output_a)
        output_b = self.base3(output_b)
        output_a = self.base4(output_a)
        output_b = self.base4(output_b)
        output_a = self.base5(output_a)
        output_b = self.base5(output_b)
        output_a = output_a.view(-1, SEQ_LEN*DIM)
        output_b = output_b.view(-1, SEQ_LEN*DIM)
        output_a = self.linear(output_a)
        output_b = self.linear(output_b)
        return output_a, output_b

class Generator(nn.Module):

    def __init__(self, charmap1, charmap2):
        super(Generator, self).__init__()

        self.fc1 = nn.Linear(128, DIM*SEQ_LEN)
        self.base1 = ResBlock()
        self.base2 = ResBlock()
        self.base3 = ResBlock()
        self.base4 = ResBlock()
        self.block5_a = ResBlock()
        self.block5_b = ResBlock()        
        self.conv1_a = nn.Conv1d(DIM, len(charmap1), 1)
        self.conv1_b = nn.Conv1d(DIM, len(charmap2), 1)
        self.softmax_a = nn.Softmax()
        self.softmax_b = nn.Softmax()

    def forward(self, noise):
        output_a = self.fc1(noise)
        output_b = self.fc1(noise)
        output_a = output_a.view(-1, DIM, SEQ_LEN) # (BATCH_SIZE, DIM, SEQ_LEN)
        output_b = output_b.view(-1, DIM, SEQ_LEN)
        output_a = self.base1(output_a)
        output_b = self.base1(output_b)
        output_a = self.base2(output_a)
        output_b = self.base2(output_b)
        output_a = self.base3(output_a)
        output_b = self.base3(output_b)
        output_a = self.base4(output_a)
        output_b = self.base4(output_b)
        output_a = self.conv1_a(self.block5_a(output_a)).transpose(1,2)
        output_b = self.conv1_b(self.block5_b(output_b)).transpose(1,2)
        shape_a = output_a.size()
        shape_b = output_b.size()
        output_a = self.softmax_a(output_a.contiguous().view(BATCH_SIZE*SEQ_LEN, -1))
        output_b = self.softmax_b(output_b.contiguous().view(BATCH_SIZE*SEQ_LEN, -1))
        return output_a.view(shape_a), output_b.view(shape_b)

In [13]:
weights = []

for param in model.parameters():
    weights.append(param.clone())

criterion = nn.BCELoss() # criterion and optimizer setup
optimizer = optim.Adam(model.parameters(), lr=0.001)

foo = torch.randn(3, 10) # fake input
target = torch.randn(3, 5) # fake target

result = model(foo) # predictions and comparison and backprop
loss = criterion(result, target)
optimizer.zero_grad()
loss.backward()
optimizer.step()


weights_after_backprop = [] # weights after backprop
for param in model.parameters():
    weights_after_backprop.append(param.clone()) # only layer1's weight should update, layer2 is not used

for i in zip(weights, weights_after_backprop):
    print(torch.equal(i[0], i[1]))

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init

# Real off-the-shelf tied linear module
class TiedLinear(nn.Module):
    def __init__(self, tied_to: nn.Linear, bias: bool = True):
        super().__init__()
        self.tied_to = tied_to
        if bias:
            self.bias = nn.Parameter(torch.Tensor(tied_to.in_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    # copied from nn.Linear
    def reset_parameters(self):
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.tied_to.weight.t())
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        return F.linear(input, self.tied_to.weight.t(), self.bias)

    # To keep module properties intuitive
    @property
    def weight(self) -> torch.Tensor:
        return self.tied_to.weight.t()

# Shared weights, different biases
encoder = nn.Linear(in, out)
decoder = TiedLinear(encoder)

SyntaxError: invalid syntax (<ipython-input-1-6cd296ef3051>, line 33)

In [7]:
import numpy as np
import torch.nn as nn 
import torch
from collections import OrderedDict
import torch.nn.functional as F

feature_ex = nn.Sequential(OrderedDict([('conv1', nn.Conv2d(1, 6, 5)),
                                        ('relu1', nn.ReLU()),
                                        ('maxpool1', nn.MaxPool2d((2, 2))),
                                        ('conv2', nn.Conv2d(6, 16, 5)),
                                        ('relu2', nn.ReLU()),
                                        ('maxpool2', nn.MaxPool2d(2))
                                        ]))


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = feature_ex(x)   # [1]
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)     # [2]

        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:       # Get the products
            num_features *= s
        return num_features


model1 = Net()
model2 = Net()

img = torch.randn(10, 1, 32, 32)
out1 = model1.forward(img)
out2 = model2.forward(img)

# [1]
# print(np.allclose(out1.detach().numpy(), out2.detach().numpy()))
# output: True

# [2]
print(np.allclose(out1.detach().numpy(), out2.detach().numpy()))

False


In [3]:
# convert images files into folder as per image name 
from pathlib import Path
import os
import shutil

MAX_FILES_PER_DIR = 100    

pngDirectory = Path('/home/parth/Documents/test_imagesimilarity/training_copy/')
pngFiles = pngDirectory.glob('*.png')   #find all png files in directory
for pngFile in pngFiles:
    fileNumber = pngFile.name.split('.')[0]         #get number from filename
  
    currentFolder = pngDirectory / "{}".format(fileNumber)
    if os.path.isdir(fileNumber) :
       pass
    else:
        currentFolder.mkdir()
        shutil.copy(pngFile,currentFolder)